In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import pandas as pd

# Compute absolute path to the directory you want to import from
sys.path.append(os.path.abspath(".."))

import main

b = 3
data_path = "../data/IMS/1st_test/"
data = f"features_scaled_{b}.csv"

df = pd.read_csv(os.path.join(data_path, data))

X_train, X_test, X_val = main.test_train_data(df)


history = main.create_model(X_train, X_test)



plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()


ValueError: too many values to unpack (expected 2)